In [1]:
#importing the package
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk import TreebankWordTokenizer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from emoji import UNICODE_EMOJI
import emoji as emoj
import demoji
from gensim import models
from gensim.models import word2vec
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import pickle
from sklearn.multiclass import OneVsRestClassifier

#### Download and install the Following

In [3]:
#nltk.download('wordnet')
#nltk.download('stopwords')
#!pip install emoji
#!pip install demoji
#demoji.download_codes()
#demoji.download_codes()
# pip install plotly 
#install genism on the ec2 instance import subprocess then , subprocess.check_call(["python",'-m','pip','install','gensim'])

In [2]:
#importing the data
data = pd.read_csv('train.csv')
data1 = data
data.head()

,sentiment,message,tweetid
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553
1,-1,All the biggest lies about climate change and ...,925046776553529344
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504


In [3]:
#checking columns
data.columns

Index(['sentiment', 'message', 'tweetid'], dtype='object')

In [4]:
data.shape

(30759, 3)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30759 entries, 0 to 30758
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  30759 non-null  int64 
 1   message    30759 non-null  object
 2   tweetid    30759 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 721.0+ KB


In [6]:
data.describe()

,sentiment,tweetid
count,30759.000000,3.075900e+04
mean,0.853929,8.364330e+17
std,0.853551,8.519986e+16
min,-1.000000,5.926334e+17
25%,0.000000,7.970301e+17
50%,1.000000,8.401516e+17
75%,1.000000,8.996975e+17
max,2.000000,9.666876e+17


In [7]:
data[['message']].describe()

,message
count,30759
unique,28813
top,RT @StephenSchlegel: she's thinking about how ...
freq,361


In [8]:
data.nunique()

sentiment        4
message      28813
tweetid      30759
dtype: int64

### Data Cleaning

### Finding Rows with emoji

In [9]:
def text_has_emoji(text):
    """This function checks all the rows of data to see if we have any emojis in the """
    for character in text:
        if character in emoj.UNICODE_EMOJI:
            return True
    return False
data['emoji'] = data['message'].apply(text_has_emoji)


We see below that the message column has 1015 rows that have emojis

In [10]:
data[data['emoji'] == True]

,sentiment,message,tweetid,emoji
7,-1,RT @DBloom451: EPA chief Pruitt rightly points...,840025826418667522,True
25,-1,RT @OffGridMedia: RIP: Weather Channel Founder...,953882558265098240,True
44,-1,RT @COLRICHARDKEMP: Earthâ€™s Magnetic Poles S...,959174163515588608,True
71,-1,RT @texas_bourne: Woke up to more snow in Texa...,955611260673912833,True
107,-1,RT @AIIAmericanGirI: 🇺🇸\nGlobal Warming FAIL! ...,692019221245526017,True
...,...,...,...,...
30354,2,"Yes, global warming will be bad. But these sci...",951548359776657408,True
30595,2,"[♥ #JuniorOrteqa ♥] In Alaska, Obama to Walk F...",637684719396614144,True
30659,2,"RT @grist: In Pruittâ€™s world, climate change...",961601013508722688,True
30682,2,â€˜Silver bulletâ€™ to suck CO2 from air and h...,957811099646050304,True


#### Extract All The Emojis In The Dataframe

In [11]:
def extra_all_the_emoji(strings):
    """We extract the emojis for every row in the dataframe"""
    return ''.join(character for character in strings if character in emoj.UNICODE_EMOJI)

We apply the extra_all_the_emoji function to the message column

In [12]:
"""We see all the emojis in the dataframe"""
data['emojis'] = data['message'].apply(extra_all_the_emoji)
data[data['emojis'] != '']['emojis']

7           ☀
25         ™™
44         ™™
71          ™
107        🇺🇸
         ... 
30354       ™
30595    ♥♥♥♥
30659     ™™™
30682       ™
30751       ⚡
Name: emojis, Length: 1015, dtype: object

In [13]:
"""The list of all emojis"""
list_of_emojis = data[data['emojis'] != '']['emojis'].tolist()

In [16]:
emoji_df = pd.DataFrame(list_of_emojis)

In [18]:
emoji_df.to_csv("emoji.csv", header=None, index=None)

In [17]:
"""We loop through the whole list to change every emoji to text """
changed_emoji_to_text = []
for emojis in list_of_emojis:
    changed_emoji_to_text.append(emoj.demojize(emojis, delimiters=("", "")))
changed_emoji_to_text    

['sun',
 'trade_marktrade_mark',
 'trade_marktrade_mark',
 'trade_mark',
 'United_States',
 'trade_mark',
 'face_with_tears_of_joyface_with_tears_of_joy',
 'United_StatesUnited_States',
 'zipper-mouth_facezipper-mouth_face',
 'party_popperthinking_faceface_with_tears_of_joyface_with_tears_of_joy',
 'trade_mark',
 'trade_markcopyright',
 'fire',
 'trade_mark',
 'squinting_face_with_tongueface_with_tongueface_with_tongue',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'oncoming_fist_light_skin_tonemiddle_finger_light_skin_tone',
 'trade_marktrade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'sparklersparkler',
 'trade_mark',
 'trade_mark',
 'trade_marktrade_mark',
 'trade_marktrade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'snowflake',
 'trade_mark',
 'nail_polish_medium_skin_tone',
 'left_arrow',
 'trade_marktrade_mark',
 'grinning_facegrinning_face_with_sweat',
 '

Now since we changed the emojis to text in a list we change them in the dataframe using the function below

In [18]:
def emojis_to_text(text):
    """We change all the emojis in the message column into words"""
    return emoj.demojize(text, delimiters=("", ""))
data['message'] = data['message'].apply(emojis_to_text)

#### Removing Twitter Handles

In [19]:
def remove_twitter_handles(tweet, pattern):
    """This function is used to remove twitter handles"""
    r = re.findall(pattern, tweet)
    for text in r:
        tweet = re.sub(text, '', tweet)
    return tweet

data['clean_tweet'] = np.vectorize(remove_twitter_handles)(data['message'], "@[\w]*") 

In [20]:
data.head()

,sentiment,message,tweetid,emoji,emojis,clean_tweet
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,False,,RT : Funny as hell! Canada demands 'gender rig...
1,-1,All the biggest lies about climate change and ...,925046776553529344,False,,All the biggest lies about climate change and ...
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,False,,The Coming Revelation Of The $q$Global Warming...
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,False,,RT : Let's see if the world ends when 's clim...
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,False,,RT : Obama has no control over the climate. He...


We use value counts to see if we stil have any emojis in the dataframe

In [21]:
data['emoji'].value_counts()

False    29744
True      1015
Name: emoji, dtype: int64

Removing Stopwords From the DataFrame

In [22]:
stop_words = nltk.corpus.stopwords.words('english')

In [23]:
data['tidy_tweet'] = data['clean_tweet'].apply(lambda x: ' '.join([w for w in x.split() if w not in stop_words]))

In [24]:
data.head()

,sentiment,message,tweetid,emoji,emojis,clean_tweet,tidy_tweet
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,False,,RT : Funny as hell! Canada demands 'gender rig...,RT : Funny hell! Canada demands 'gender rights...
1,-1,All the biggest lies about climate change and ...,925046776553529344,False,,All the biggest lies about climate change and ...,All biggest lies climate change global warming...
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,False,,The Coming Revelation Of The $q$Global Warming...,The Coming Revelation Of The $q$Global Warming...
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,False,,RT : Let's see if the world ends when 's clim...,RT : Let's see world ends 's climate change ro...
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,False,,RT : Obama has no control over the climate. He...,RT : Obama control climate. He worst snake oil...


Remove http URL from data

In [25]:
data['tidy_tweet'] = data['tidy_tweet'].apply(lambda x: re.split('https:\/\/.*',str(x))[0])
data['tidy_tweet'] 

0        RT : Funny hell! Canada demands 'gender rights...
1        All biggest lies climate change global warming...
2        The Coming Revelation Of The $q$Global Warming...
3        RT : Let's see world ends 's climate change ro...
4        RT : Obama control climate. He worst snake oil...
                               ...                        
30754    RT : The Pentagon warned climate change threat...
30755    Study finds global warming exacerbates refugee...
30756    RT : The global green movement prepares fight ...
30757    RT : NYC Mayor says NY go faster address clima...
30758         RT Sanders: Climate Change Causes Terrorism 
Name: tidy_tweet, Length: 30759, dtype: object

Text Normalization

In [26]:
def tokenizing(text):
    text = re.split('\W+', text)
    return text

data['tokenized_tweet'] = data['tidy_tweet'].apply(lambda x: tokenizing(x))

In [27]:
data.head()

,sentiment,message,tweetid,emoji,emojis,clean_tweet,tidy_tweet,tokenized_tweet
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,False,,RT : Funny as hell! Canada demands 'gender rig...,RT : Funny hell! Canada demands 'gender rights...,"[RT, Funny, hell, Canada, demands, gender, rig..."
1,-1,All the biggest lies about climate change and ...,925046776553529344,False,,All the biggest lies about climate change and ...,All biggest lies climate change global warming...,"[All, biggest, lies, climate, change, global, ..."
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,False,,The Coming Revelation Of The $q$Global Warming...,The Coming Revelation Of The $q$Global Warming...,"[The, Coming, Revelation, Of, The, q, Global, ..."
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,False,,RT : Let's see if the world ends when 's clim...,RT : Let's see world ends 's climate change ro...,"[RT, Let, s, see, world, ends, s, climate, cha..."
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,False,,RT : Obama has no control over the climate. He...,RT : Obama control climate. He worst snake oil...,"[RT, Obama, control, climate, He, worst, snake..."


In [28]:
tokens = data['tokenized_tweet']

In [29]:
lemmatizer = WordNetLemmatizer()

In [30]:
tokens = tokens.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])

In [31]:
data['lemmatized_tweet'] = tokens

In [32]:
data.head()

,sentiment,message,tweetid,emoji,emojis,clean_tweet,tidy_tweet,tokenized_tweet,lemmatized_tweet
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,False,,RT : Funny as hell! Canada demands 'gender rig...,RT : Funny hell! Canada demands 'gender rights...,"[RT, Funny, hell, Canada, demands, gender, rig...","[RT, Funny, hell, Canada, demand, gender, righ..."
1,-1,All the biggest lies about climate change and ...,925046776553529344,False,,All the biggest lies about climate change and ...,All biggest lies climate change global warming...,"[All, biggest, lies, climate, change, global, ...","[All, biggest, lie, climate, change, global, w..."
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,False,,The Coming Revelation Of The $q$Global Warming...,The Coming Revelation Of The $q$Global Warming...,"[The, Coming, Revelation, Of, The, q, Global, ...","[The, Coming, Revelation, Of, The, q, Global, ..."
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,False,,RT : Let's see if the world ends when 's clim...,RT : Let's see world ends 's climate change ro...,"[RT, Let, s, see, world, ends, s, climate, cha...","[RT, Let, s, see, world, end, s, climate, chan..."
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,False,,RT : Obama has no control over the climate. He...,RT : Obama control climate. He worst snake oil...,"[RT, Obama, control, climate, He, worst, snake...","[RT, Obama, control, climate, He, worst, snake..."


In [33]:
data = data.drop(['tidy_tweet'],axis=1)
data = data.drop(['tokenized_tweet'], axis=1)

In [34]:
data.head()

,sentiment,message,tweetid,emoji,emojis,clean_tweet,lemmatized_tweet
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,False,,RT : Funny as hell! Canada demands 'gender rig...,"[RT, Funny, hell, Canada, demand, gender, righ..."
1,-1,All the biggest lies about climate change and ...,925046776553529344,False,,All the biggest lies about climate change and ...,"[All, biggest, lie, climate, change, global, w..."
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,False,,The Coming Revelation Of The $q$Global Warming...,"[The, Coming, Revelation, Of, The, q, Global, ..."
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,False,,RT : Let's see if the world ends when 's clim...,"[RT, Let, s, see, world, end, s, climate, chan..."
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,False,,RT : Obama has no control over the climate. He...,"[RT, Obama, control, climate, He, worst, snake..."


In [35]:
clean_data = data[['sentiment','message','lemmatized_tweet','clean_tweet','tweetid']]

In [36]:
clean_data.head()

,sentiment,message,lemmatized_tweet,clean_tweet,tweetid
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,"[RT, Funny, hell, Canada, demand, gender, righ...",RT : Funny as hell! Canada demands 'gender rig...,897853122080407553
1,-1,All the biggest lies about climate change and ...,"[All, biggest, lie, climate, change, global, w...",All the biggest lies about climate change and ...,925046776553529344
2,-1,The Coming Revelation Of The $q$Global Warming...,"[The, Coming, Revelation, Of, The, q, Global, ...",The Coming Revelation Of The $q$Global Warming...,696354236850786305
3,-1,RT @DineshDSouza: Let's see if the world ends ...,"[RT, Let, s, see, world, end, s, climate, chan...",RT : Let's see if the world ends when 's clim...,846806509732483072
4,-1,RT @SteveSGoddard: Obama has no control over t...,"[RT, Obama, control, climate, He, worst, snake...",RT : Obama has no control over the climate. He...,628085266293653504


In [37]:
clean_data.to_csv("clean_data.csv",index=False)

### Visualizing Tweets with Word2Vec and t-SNE

In [38]:
data_v = data1.copy()
data_v.head()

,sentiment,message,tweetid,emoji,emojis,clean_tweet,tidy_tweet,tokenized_tweet,lemmatized_tweet
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,False,,RT : Funny as hell! Canada demands 'gender rig...,RT : Funny hell! Canada demands 'gender rights...,"[RT, Funny, hell, Canada, demands, gender, rig...","[RT, Funny, hell, Canada, demand, gender, righ..."
1,-1,All the biggest lies about climate change and ...,925046776553529344,False,,All the biggest lies about climate change and ...,All biggest lies climate change global warming...,"[All, biggest, lies, climate, change, global, ...","[All, biggest, lie, climate, change, global, w..."
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,False,,The Coming Revelation Of The $q$Global Warming...,The Coming Revelation Of The $q$Global Warming...,"[The, Coming, Revelation, Of, The, q, Global, ...","[The, Coming, Revelation, Of, The, q, Global, ..."
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,False,,RT : Let's see if the world ends when 's clim...,RT : Let's see world ends 's climate change ro...,"[RT, Let, s, see, world, ends, s, climate, cha...","[RT, Let, s, see, world, end, s, climate, chan..."
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,False,,RT : Obama has no control over the climate. He...,RT : Obama control climate. He worst snake oil...,"[RT, Obama, control, climate, He, worst, snake...","[RT, Obama, control, climate, He, worst, snake..."


In [39]:
STOP_WORDS = nltk.corpus.stopwords.words()

def clean_sentence(val):
    "remove chars that are not letters or numbers, downcase, then remove stop words"
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = sentence.split(" ")
    
    for word in list(sentence):
        if word in STOP_WORDS:
            sentence.remove(word)  
            
    sentence = " ".join(sentence)
    return sentence

def clean_dataframe(data):
    "drop nans, then apply 'clean_sentence' function to question1 and 2"
    data = data.dropna(how="any")
    
    for col in ['message']:
        data[col] = data[col].apply(clean_sentence)
    
    return data
data_v = clean_dataframe(data_v)
data_v.head(5)

,sentiment,message,tweetid,emoji,emojis,clean_tweet,tidy_tweet,tokenized_tweet,lemmatized_tweet
0,-1,rt darreljorstad funny hell canada demands gen...,897853122080407553,False,,RT : Funny as hell! Canada demands 'gender rig...,RT : Funny hell! Canada demands 'gender rights...,"[RT, Funny, hell, Canada, demands, gender, rig...","[RT, Funny, hell, Canada, demand, gender, righ..."
1,-1,biggest lies climate change global warming deb...,925046776553529344,False,,All the biggest lies about climate change and ...,All biggest lies climate change global warming...,"[All, biggest, lies, climate, change, global, ...","[All, biggest, lie, climate, change, global, w..."
2,-1,coming revelation qglobal warmingq fraud resem...,696354236850786305,False,,The Coming Revelation Of The $q$Global Warming...,The Coming Revelation Of The $q$Global Warming...,"[The, Coming, Revelation, Of, The, q, Global, ...","[The, Coming, Revelation, Of, The, q, Global, ..."
3,-1,rt dineshdsouza lets see world ends realdonald...,846806509732483072,False,,RT : Let's see if the world ends when 's clim...,RT : Let's see world ends 's climate change ro...,"[RT, Let, s, see, world, ends, s, climate, cha...","[RT, Let, s, see, world, end, s, climate, chan..."
4,-1,rt stevesgoddard obama control climate worst s...,628085266293653504,False,,RT : Obama has no control over the climate. He...,RT : Obama control climate. He worst snake oil...,"[RT, Obama, control, climate, He, worst, snake...","[RT, Obama, control, climate, He, worst, snake..."


In [40]:
def build_corpus(data):
    "Creates a list of lists containing words from each sentence"
    corpus = []
    for col in ['message']:
        for sentence in data[col].iteritems():
            word_list = sentence[1].split(" ")
            corpus.append(word_list)
            
    return corpus

corpus = build_corpus(data_v)        
corpus[0:2]

[['rt',
  'darreljorstad',
  'funny',
  'hell',
  'canada',
  'demands',
  'gender',
  'rights',
  'climate',
  'change',
  'trade',
  'deal',
  'soviet',
  'dairy',
  'boards',
  'untouc'],
 ['biggest',
  'lies',
  'climate',
  'change',
  'global',
  'warming',
  'debunked',
  'astonishing',
  'interview',
  'httpstcongvipo4wya']]

In [41]:
model = word2vec.Word2Vec(corpus, size=100, window=20, min_count=200, workers=4)
model.wv['warming']

array([ 0.14356254, -0.04737179, -0.20461252, -0.09623345, -0.3150102 ,
        0.22960633,  0.33925322,  0.1563711 , -0.00906569, -0.3105211 ,
        0.7908593 , -0.40081447,  0.52098185,  0.18941133, -0.25930417,
       -0.67550576,  0.29557118,  0.07352812, -0.6422742 ,  0.17723574,
        0.08867937, -0.11805269,  0.6630559 ,  0.8010343 , -0.3513215 ,
       -0.01892206,  0.51123494,  0.51339966,  0.5369584 , -0.23578636,
        0.06837638,  0.4106011 , -0.18675302, -0.04789706, -0.22555974,
       -0.04215088, -0.41678208,  0.36924857,  0.32986698, -0.19857794,
       -0.23331259,  0.14977501, -0.47920236, -0.05941273, -0.0727421 ,
        0.5540072 ,  0.11253476,  0.50776637, -0.27150252, -0.01495708,
       -0.20311555,  0.30457363, -0.38774756, -0.29618692,  0.23035304,
       -0.54150075,  0.10760351, -0.24898262,  0.28425023, -0.47776142,
       -0.685266  ,  0.71096325,  0.29854536, -0.366499  , -0.3717186 ,
        0.5359599 ,  0.06471889,  0.23831683,  0.36912093, -0.50

In [42]:
model.wv['warming']

array([ 0.14356254, -0.04737179, -0.20461252, -0.09623345, -0.3150102 ,
        0.22960633,  0.33925322,  0.1563711 , -0.00906569, -0.3105211 ,
        0.7908593 , -0.40081447,  0.52098185,  0.18941133, -0.25930417,
       -0.67550576,  0.29557118,  0.07352812, -0.6422742 ,  0.17723574,
        0.08867937, -0.11805269,  0.6630559 ,  0.8010343 , -0.3513215 ,
       -0.01892206,  0.51123494,  0.51339966,  0.5369584 , -0.23578636,
        0.06837638,  0.4106011 , -0.18675302, -0.04789706, -0.22555974,
       -0.04215088, -0.41678208,  0.36924857,  0.32986698, -0.19857794,
       -0.23331259,  0.14977501, -0.47920236, -0.05941273, -0.0727421 ,
        0.5540072 ,  0.11253476,  0.50776637, -0.27150252, -0.01495708,
       -0.20311555,  0.30457363, -0.38774756, -0.29618692,  0.23035304,
       -0.54150075,  0.10760351, -0.24898262,  0.28425023, -0.47776142,
       -0.685266  ,  0.71096325,  0.29854536, -0.366499  , -0.3717186 ,
        0.5359599 ,  0.06471889,  0.23831683,  0.36912093, -0.50

In [43]:
model_to = pd.DataFrame(model.wv['warming']).to_csv("model1.csv", header=None, index=None)

In [44]:
m1 = pd.read_csv("model1.csv")
m1

,0.14356254
0,-0.047372
1,-0.204613
2,-0.096233
3,-0.315010
4,0.229606
...,...
94,0.438816
95,-0.291004
96,0.556329
97,0.088950


In [45]:
import numpy
numpy.savetxt("model.csv", model.wv['warming'], delimiter=",")

In [27]:
m = pd.read_csv("model.csv")

In [31]:
np.array(m)

99

### We want to pickle our models to use for classifying Tweets

In [46]:
# split data into X and y
X = clean_data['lemmatized_tweet']
y = clean_data['sentiment']

### Split data 

In [47]:
# get training and testing data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
X_train = X_train.str.join(', ')

In [49]:
X_test = X_test.str.join(', ')

### Vectorize the data 

In [50]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [51]:
#pipeline of feature engineering and model
model = Pipeline([('vectorizer', CountVectorizer(lowercase=False)),
 ('tfidf', TfidfTransformer()),
 ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced')))])
#the class_weight="balanced" option tries to remove the biasedness of model towards majority sample

In [52]:
#paramater selection
from sklearn.model_selection import GridSearchCV
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}
gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

0.7058971241759582
{'tfidf__use_idf': True, 'vectorizer__ngram_range': (1, 2)}


In [53]:
#preparing the final pipeline using the selected parameters
model = Pipeline([('vectorizer', CountVectorizer(lowercase=False,ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

In [54]:
#fit model with training data
model.fit(X_train, y_train)
#evaluation on test data
pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(pred, y_test)

array([[ 269,   80,   95,   20],
       [  95,  528,  201,   59],
       [ 122,  381, 2643,  242],
       [  30,  106,  315,  966]], dtype=int64)

In [55]:
model_save_path = "Linear_Support1.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(model,file)

### Model fitting and evaluation

Linear Support Vector

In [56]:
# define the model 
linear_support_vector = LinearSVC()
# fit the model 
linear_support_vector.fit(X_train_tfidf, y_train)

text_classifier = Pipeline([
    ('tfidf', TfidfVectorizer()),  # integer counts to weighted TF-IDF scores
    ('classifier',LinearSVC()),  # train on TF-IDF vectors w/ Linear Support Vector Classifier
])
# Feed the training data through the pipeline
text_classifier.fit(X_train, y_train) 
# get predictions on the test set 
y_pred = text_classifier.predict(X_test)
y_pred

array([1, 1, 2, ..., 1, 1, 1], dtype=int64)

### Saving a model with Pickle

In [65]:
model_save_path = "Linear_Support_Vector_Classifier_model.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(text_classifier,file)

In [63]:
X_test.dtype

dtype('O')

In [66]:
text_classifier.predict(pd.Series(X_test.iloc[0]))

array([1], dtype=int64)